In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from scipy.io import loadmat
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/cycles_50_final_dataset.csv', infer_datetime_format=True)

In [3]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [4]:
dataset = df[df["State"] == 2]

In [5]:
dataset.reset_index(drop=True, inplace=True)

In [6]:
dataset

,BSM-p,BSM-m,BSM-c,Datetime,Voltage-p,Voltage-m,Voltage-c,Current,Temperature-p,Temperature-m,State,Capacity,SOC,SOH,Cycles
0,SN-P10001,SN-M10001,SN-C10001,2008-04-02 15:25:41,372.665516,49.383190,3.795266,62.221067,31.237017,30.447017,2,1.856487,100.0,1.000000,1
1,SN-P10001,SN-M10001,SN-C10002,2008-04-02 15:25:41,372.665516,49.383190,4.445266,62.221067,31.237017,30.447017,2,1.856487,100.0,1.000000,1
2,SN-P10001,SN-M10001,SN-C10003,2008-04-02 15:25:41,372.665516,49.383190,4.495266,62.221067,31.237017,30.447017,2,1.856487,100.0,1.000000,1
3,SN-P10001,SN-M10001,SN-C10004,2008-04-02 15:25:41,372.665516,49.383190,3.555266,62.221067,31.237017,30.447017,2,1.856487,100.0,1.000000,1
4,SN-P10001,SN-M10001,SN-C10005,2008-04-02 15:25:41,372.665516,49.383190,3.975266,62.221067,31.237017,30.447017,2,1.856487,100.0,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210747,SN-P10001,SN-M10008,SN-C10092,2008-04-29 22:00:04,363.222733,45.062842,3.455237,62.126966,36.590995,37.340995,2,1.767364,NaN,0.951994,50
1210748,SN-P10001,SN-M10008,SN-C10093,2008-04-29 22:00:04,363.222733,45.062842,3.065237,62.126966,36.590995,37.340995,2,1.767364,NaN,0.951994,50
1210749,SN-P10001,SN-M10008,SN-C10094,2008-04-29 22:00:04,363.222733,45.062842,4.535237,62.126966,36.590995,37.340995,2,1.767364,NaN,0.951994,50
1210750,SN-P10001,SN-M10008,SN-C10095,2008-04-29 22:00:04,363.222733,45.062842,3.685237,62.126966,36.590995,37.340995,2,1.767364,NaN,0.951994,50


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

In [8]:
# 원하는 컬럼 추출
selected_columns = ['Cycles', 'Datetime', 'Capacity']
df_selected = dataset[selected_columns]

# 중복 제거
capacity_val = df_selected.drop_duplicates()

In [9]:
attrib=['Cycles', 'Datetime', 'Capacity']

In [10]:
dis_ele = capacity_val[attrib]
rows=['Cycles','Capacity']
dataset=dis_ele[rows]
data_train=dataset[(dataset['Cycles']<50)]
data_set_train=data_train.iloc[:,1:2].values
data_test=dataset[(dataset['Cycles']>=50)]
data_set_test=data_test.iloc[:,1:2].values

In [11]:
sc=MinMaxScaler(feature_range=(0,1))
data_set_train=sc.fit_transform(data_set_train)
data_set_test=sc.transform(data_set_test)

In [12]:
X_train=[]
y_train=[]

In [13]:
#take the last 10t to predict 10t+1
for i in range(10,49):
    X_train.append(data_set_train[i-10:i,0])
    y_train.append(data_set_train[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)

In [14]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [17]:
regress = Sequential()
regress.add(LSTM(units=200, return_sequences=True, input_shape=(X_train.shape[1],1)))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200, return_sequences=True))
regress.add(Dropout(0.3))
regress.add(LSTM(units=200))
regress.add(Dropout(0.3))
regress.add(Dense(units=1))
regress.compile(optimizer='adam',loss='mean_squared_error')
regress.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 200)           161600    
                                                                 
 dropout (Dropout)           (None, 10, 200)           0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 200)           320800    
                                                                 
 dropout_1 (Dropout)         (None, 10, 200)           0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 200)           320800    
                                                                 
 dropout_2 (Dropout)         (None, 10, 200)           0         
                                                                 
 lstm_3 (LSTM)               (None, 200)               3

In [18]:
regress.fit(X_train,y_train,epochs=200,batch_size=25)

Epoch 1/200
2/2 [==============================] - 7s 44ms/step - loss: 0.3260
Epoch 2/200
2/2 [==============================] - 0s 39ms/step - loss: 0.1289
Epoch 3/200
2/2 [==============================] - 0s 40ms/step - loss: 0.1183
Epoch 4/200
2/2 [==============================] - 0s 41ms/step - loss: 0.0481
Epoch 5/200
2/2 [==============================] - 0s 39ms/step - loss: 0.0769
Epoch 6/200
2/2 [==============================] - 0s 40ms/step - loss: 0.0620
Epoch 7/200
2/2 [==============================] - 0s 39ms/step - loss: 0.0462
Epoch 8/200
2/2 [==============================] - 0s 42ms/step - loss: 0.0608
Epoch 9/200
2/2 [==============================] - 0s 42ms/step - loss: 0.0469
Epoch 10/200
2/2 [==============================] - 0s 40ms/step - loss: 0.0397
Epoch 11/200
2/2 [==============================] - 0s 41ms/step - loss: 0.0561
Epoch 12/200
2/2 [==============================] - 0s 42ms/step - loss: 0.0521
Epoch 13/200
2/2 [==============================]

In [20]:
print(len(data_test))
data_total=pd.concat((data_train['Capacity'], data_test['Capacity']),axis=0)
inputs=data_total[len(data_total)-len(data_test)-10:].values
inputs=inputs.reshape(-1,1)
inputs=sc.transform(inputs)

1


In [21]:
X_test=[]
for i in range(10,129):
    X_test.append(inputs[i-10:i,0])
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
pred=regress.predict(X_test)
print(pred.shape)
pred=sc.inverse_transform(pred)
pred=pred[:,0]
tests=data_test.iloc[:,1:2]
rmse = np.sqrt(mean_squared_error(tests, pred))
print('Test RMSE: %.3f' % rmse)
metrics.r2_score(tests,pred)

IndexError: tuple index out of range